# 🔌 Model Context Protocol (MCP) & Multi-Agent Workflows

This notebook explains the **Model Context Protocol (MCP)**, how to implement an MCP server, and how to structure it within a **multi-agent architecture** using the Anthropic SDK.

| Section | What you'll learn |
|---|---|
| **1** | What is MCP? |
| **2** | Installation & Setup |
| **3** | Building an MCP Server (Excel Tools) |
| **4** | Creating Sample Data (Simulating an Upload) |
| **5** | Connecting the Client & Converting Tools |
| **6** | Multi-Agent Architecture (Analyst & Editor) |
| **7** | Running the Full Workflow |
| **8** | Proposed Project Structure for Production |

---

### 1 · What is MCP?

Think of MCP as the **"USB-C for AI agents"**. 

Historically, if you wanted an AI to read a database, search the web, and edit an Excel file, you had to write custom integration code for each tool. 

**MCP standardizes this:**
- **MCP Server:** A lightweight program that exposes tools, resources, and prompts using a standard protocol. (e.g., an "Excel MCP Server").
- **MCP Client:** An AI agent (like Claude Desktop, or your custom Python script) that connects to the server, discovers its tools, and asks the server to execute them.

In a **Multi-Agent Architecture**, MCP servers act as the "capabilities layer". You can have a single MCP server exposing 50 tools, and give different specialized agents access to different subsets of those tools.

---
## 2 · Installation & Setup

In [ ]:
# Install the required packages
%pip install mcp anthropic pandas openpyxl --quiet

In [ ]:
import os

# Set your Anthropic API key
# os.environ["ANTHROPIC_API_KEY"] = "sk-ant-..."

assert os.environ.get("ANTHROPIC_API_KEY"), "⚠️  Please set ANTHROPIC_API_KEY before continuing."
print("✅ API key is set.")

---
## 3 · Building an MCP Server (Excel Tools)

We will build an MCP server that exposes two tools:
1. `read_excel_data`: Reads an Excel file and returns the data (including row indices).
2. `update_excel_cell`: Updates a specific cell in the Excel file.

Because MCP servers run as separate processes (usually communicating via standard input/output), we will write this server to a separate Python file using the `%%writefile` magic command.

In [ ]:
%%writefile excel_mcp_server.py
from mcp.server.fastmcp import FastMCP
import pandas as pd

# Initialize the FastMCP server
mcp = FastMCP("Excel Server")

@mcp.tool()
def read_excel_data(file_path: str) -> str:
    """Reads an Excel file and returns its contents as a JSON string. 
    Includes the 'index' which is required for updating cells later.
    """
    try:
        df = pd.read_excel(file_path)
        # Reset index so the LLM can see the row numbers
        df_reset = df.reset_index()
        return df_reset.to_json(orient="records")
    except Exception as e:
        return f"Error reading Excel file: {e}"

@mcp.tool()
def update_excel_cell(file_path: str, row_index: int, column_name: str, new_value: str) -> str:
    """Updates a specific cell in an Excel file and saves it.
    
    Args:
        file_path: Path to the Excel file.
        row_index: The integer index of the row to update (obtained from read_excel_data).
        column_name: The name of the column to update.
        new_value: The new value to insert.
    """
    try:
        df = pd.read_excel(file_path)
        
        # Attempt to convert numeric strings to actual numbers
        try:
            new_value = float(new_value)
        except ValueError:
            pass
            
        df.at[row_index, column_name] = new_value
        df.to_excel(file_path, index=False)
        return f"✅ Successfully updated row {row_index}, column '{column_name}' to {new_value}."
    except Exception as e:
        return f"Error updating Excel file: {e}"

if __name__ == "__main__":
    # Run the server using standard input/output
    mcp.run()

---
## 4 · Creating Sample Data (Simulating an Upload)

Let's create a dummy Excel file. In a real application, this would be the file the user uploads to your system.

In [ ]:
import pandas as pd

data = {
    "Employee": ["Alice", "Bob", "Charlie", "Diana"],
    "Department": ["Sales", "Sales", "Sales", "Sales"],
    "Sales": [450, 210, 600, 320]
}

df = pd.DataFrame(data)
df.to_excel("sales_data.xlsx", index=False)

print("✅ Created 'sales_data.xlsx'")
display(df)

---
## 5 · Connecting the Client & Multi-Agent Architecture

Now we build the **Client** side. 

### The Architecture
We will use a **Sequential Orchestrator Pattern**:
1. **Data Analyst Agent**: Gets the `read_excel_data` tool. Its job is to understand the file and find the target.
2. **Data Editor Agent**: Gets the `update_excel_cell` tool. Its job is to apply the changes based on the Analyst's findings.
3. **Orchestrator**: Manages the flow, passing the user's request to the Analyst, and then passing the Analyst's findings to the Editor.

First, let's define a generic `run_agent` function that connects Claude to our MCP session.

In [ ]:
from anthropic import AsyncAnthropic

async def run_agent(name: str, system_prompt: str, tools: list, user_input: str, mcp_session) -> str:
    """
    Runs a single agent loop. It sends the prompt to Claude, and if Claude 
    decides to use a tool, it executes the tool via the MCP session.
    """
    client = AsyncAnthropic()
    messages = [{"role": "user", "content": user_input}]
    
    print(f"\n{'='*50}")
    print(f"🤖 Starting Agent: {name}")
    print(f"{'='*50}")
    
    while True:
        response = await client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=1024,
            system=system_prompt,
            tools=tools,
            messages=messages
        )
        
        # If Claude is done, return the final text
        if response.stop_reason == "end_turn":
            final_text = next(b.text for b in response.content if b.type == "text")
            print(f"\n💬 [{name}] Final Response:\n{final_text}")
            return final_text
            
        # If Claude wants to use a tool
        elif response.stop_reason == "tool_use":
            messages.append({"role": "assistant", "content": response.content})
            tool_results = []
            
            for block in response.content:
                if block.type == "tool_use":
                    print(f"  🔧 [{name}] Calling MCP Tool: {block.name}({block.input})")
                    
                    # Execute the tool via the MCP server!
                    mcp_result = await mcp_session.call_tool(block.name, arguments=block.input)
                    result_text = mcp_result.content[0].text
                    
                    print(f"  📤 [{name}] MCP Tool Result: {result_text[:100]}...")
                    
                    tool_results.append({
                        "type": "tool_result",
                        "tool_use_id": block.id,
                        "content": result_text
                    })
            
            # Send the tool results back to Claude
            messages.append({"role": "user", "content": tool_results})
            
        else:
            print(f"Unexpected stop reason: {response.stop_reason}")
            break

---
## 6 · Running the Full Workflow

Now we tie it all together. We will:
1. Start the MCP server process.
2. Connect to it using `stdio_client`.
3. Fetch the tools and convert them to Claude's format.
4. Split the tools between our Analyst and Editor agents.
5. Run the Orchestrator workflow.

In [ ]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def orchestrator_workflow(user_request: str, mcp_session, analyst_tools: list, editor_tools: list):
    print(f"👤 User Request: {user_request}")
    
    # --- Step 1: Data Analyst Agent ---
    analyst_prompt = (
        "You are a Data Analyst. Use your tools to read and analyze the Excel file. "
        "Return a concise summary of your findings, specifically identifying the row index "
        "and details of the target mentioned in the user's request."
    )
    analyst_result = await run_agent("Data Analyst", analyst_prompt, analyst_tools, user_request, mcp_session)
    
    # --- Step 2: Data Editor Agent ---
    editor_prompt = (
        f"You are a Data Editor. The Analyst found the following:\n{analyst_result}\n\n"
        f"Based on the user's original request: '{user_request}', use your tools to update "
        "the Excel file appropriately. You MUST use the exact row index provided by the analyst."
    )
    editor_result = await run_agent("Data Editor", editor_prompt, editor_tools, user_request, mcp_session)
    
    print("\n✅ Workflow Complete! The Excel file has been updated.")


async def main():
    # 1. Define how to start the MCP server
    server_params = StdioServerParameters(
        command="python",
        args=["excel_mcp_server.py"]
    )
    
    print("🔌 Starting and connecting to Excel MCP Server...")
    
    # 2. Connect to the server
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            print("✅ Connected to MCP Server!")
            
            # 3. Fetch tools from the server
            mcp_tools = await session.list_tools()
            
            # 4. Convert MCP tools to Claude format
            claude_tools = []
            for t in mcp_tools.tools:
                claude_tools.append({
                    "name": t.name,
                    "description": t.description,
                    "input_schema": t.inputSchema
                })
            
            # 5. Distribute tools to specialized agents
            analyst_tools = [t for t in claude_tools if "read" in t["name"]]
            editor_tools = [t for t in claude_tools if "update" in t["name"]]
            
            # 6. Run the workflow
            user_request = (
                "Here is 'sales_data.xlsx'. Please analyze it, find the employee "
                "with the lowest sales, and update their sales to 500 to give them a bonus."
            )
            
            await orchestrator_workflow(user_request, session, analyst_tools, editor_tools)

# Execute the main function
await main()

### Verify the changes
Let's read the Excel file back into pandas to verify the Editor Agent successfully updated Bob's sales to 500.

In [ ]:
df_updated = pd.read_excel("sales_data.xlsx")
display(df_updated)

---
## 8 · Proposed Project Structure for Production

When building a real multi-agent MCP system, you should strictly separate your **Servers** (capabilities) from your **Clients** (agents/brains).

```text
my-mcp-project/
│
├── mcp_servers/                  # 🔌 The Capabilities Layer (No LLM logic here)
│   ├── excel_server.py           # Exposes read/write tools for Excel
│   ├── database_server.py        # Exposes SQL query tools
│   └── github_server.py          # Exposes PR/Issue tools
│
├── agents/                       # 🧠 The Brains Layer (LLM logic)
│   ├── __init__.py
│   ├── base_agent.py             # Base class handling the Claude tool-use loop
│   ├── analyst_agent.py          # System prompt for analyzing data
│   └── editor_agent.py           # System prompt for modifying data
│
├── orchestrator.py               # 🚦 The Router
│                                 # 1. Connects to MCP servers via stdio_client
│                                 # 2. Fetches tools
│                                 # 3. Passes tools to the right agents
│                                 # 4. Manages the workflow (Analyst -> Editor)
│
├── requirements.txt
└── .env                          # ANTHROPIC_API_KEY
```

### Why this structure is powerful:
1. **Security & Isolation**: The LLM never runs code directly. It only asks the MCP server to run predefined tools. You can run the MCP server in a Docker container with restricted permissions.
2. **Reusability**: You can connect Claude Desktop directly to `excel_server.py` to use the tools manually, without changing any code.
3. **Scalability**: As your system grows, you just add more MCP servers. Your agents automatically discover the new tools via `session.list_tools()`.